In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import mxnet
mxnet.__version__

'1.6.0'

In [3]:
import sys
module_path='/home/jovyan/Crop_boundary_model/'
if module_path not in sys.path:
    sys.path.append(module_path)
print(sys.path)

['/home/jovyan/Crop_boundary_model/decode/examples', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/jovyan/.local/lib/python3.8/site-packages', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.8/dist-packages/IPython/extensions', '/home/jovyan/.ipython', '/home/jovyan/Crop_boundary_model/']


In [4]:
from mxnet import nd 
from decode.FracTAL_ResUNet.models.heads.head_cmtsk import *

In [5]:
# D6nf32 example 
depth=6
norm_type='GroupNorm'
norm_groups=4
ftdepth=5
NClasses=2
nfilters_init=32
psp_depth=4
nheads_start=4


net = Head_CMTSK_BC(_nfilters_init=nfilters_init, _NClasses=NClasses,depth=depth, ftdepth=ftdepth, psp_depth=psp_depth,norm_type=norm_type,norm_groups=norm_groups,nheads_start=nheads_start)
net.initialize()

In [6]:
BatchSize = 4
img_size=256
NChannels = 3

input_img_1 = nd.random.uniform(shape=[BatchSize, NChannels, img_size, img_size])
input_img_2 = nd.random.uniform(shape=[BatchSize, NChannels, img_size, img_size])

## Example of forward operation:


In [7]:
outs = net(input_img_1, input_img_2)

In [8]:
# outs is a list of outputs, segmentation, boundary, distance. 
# Each has shape BatchSize, NClasses, img_size, img_size
for out in outs:
    print (out.shape)

(4, 2, 256, 256)
(4, 2, 256, 256)
(4, 2, 256, 256)


### Example of performing backward with multitasking operation

In [10]:
labels_segm = nd.random.uniform(shape=[BatchSize, NClasses, img_size, img_size])
labels_segm = labels_segm > 0.5

labels_bound = nd.random.uniform(shape=[BatchSize, NClasses, img_size, img_size])
labels_bound = labels_bound > 0.5

labels_dist = nd.random.uniform(shape=[BatchSize, NClasses, img_size, img_size])


labels = [labels_segm,labels_bound,labels_dist]
labels = nd.concat(*labels,dim=1)

In [12]:
from mxnet import autograd
from decode.FracTAL_ResUNet.nn.loss.mtsk_loss import *

In [13]:
myMTSKL = mtsk_loss()

In [14]:
with autograd.record():
    listOfPreds = net(input_img_1, input_img_2)
    loss = myMTSKL.loss(listOfPreds,labels)
    loss.backward()

In [15]:
loss.shape

(4,)

In [16]:
loss


[0.42095947 0.4213251  0.4212415  0.42106745]
<NDArray 4 @cpu(0)>